In [1]:
using LinearAlgebra

function compute_expectation(F, F_perp, u)
    # Extraire les blocs de la matrice u
    u_F = u[F, F]  # Bloc correspondant à F
    u_F_perp = u[F_perp, F_perp]  # Bloc correspondant à F^\perp
    u_F_Fperp = u[F, F_perp]  # Bloc reliant F et F^\perp
    u_Fperp_F = u[F_perp, F]  # Bloc reliant F^\perp et F
    
    # Calcul de l'inverse des blocs
    u_F_inv = inv(u_F)
    u_F_perp_inv = inv(u_F_perp)
    
    # Calcul de l'inverse restreint de u sur F
    u_inv_F = u_F_inv + u_F_inv * u_F_Fperp * u_F_perp_inv * u_Fperp_F * u_F_inv
    
    return u_inv_F
end

# Exemple d'utilisation
u = [4.0 1.0 0.5; 1.0 3.0 0.2; 0.5 0.2 2.0]  # Matrice symétrique
F = [1, 2]  # Indices du sous-espace F
F_perp = [3]  # Indices du sous-espace F^\perp

result = compute_expectation(F, F_perp, u)
println("Inverse restreint de u sur F:")
println(result)


Inverse restreint de u sur F:
[0.2797107438016529 -0.08929752066115702; -0.08929752066115702 0.3640082644628099]


In [104]:
using LinearAlgebra

# Inversion par Complément de Schur (méthode alternative)
function schur_inverse(A::Matrix{Float64})
    n = size(A, 1)
    if n == 1 
        return [1.0 ./ A[1]]
    elseif n == 0 
        return 0 
    end 
    
    k = div(n, 2)  # Division en blocs
    #println("k , n  , A =" , k ," , " ,  n , " , " , A )
    A11 = A[1:k, 1:k]
    A12 = A[1:k, k+1:end]
    A21 = A[k+1:end, 1:k]
    A22 = A[k+1:end, k+1:end]
    #
    println(A11 , ", " , A12 , ", " , A21 , ", " , A22 )

    inv_A11 = schur_inverse(A11)
    S = A22 - A21 * inv_A11 * A12  # Complément de Schur
    inv_S = schur_inverse(S)

    # Reconstruction de l'inverse
    inv_A = [inv_A11 + inv_A11 * A12 * inv_S * A21 * inv_A11   -inv_A11 * A12 * inv_S
             -inv_S * A21 * inv_A11                             inv_S]
    
    return inv_A
end

function compute_inverse_restricted(F, F_perp, u)
    # Extraire les blocs de la matrice u

    if F_perp == []
        return schur_inverse(u)
    end 
    u_F = u[F, F]  # Bloc correspondant à F
    u_F_perp = u[F_perp, F_perp]  # Bloc correspondant à F^\perp
    u_F_Fperp = u[F, F_perp]  # Bloc reliant F et F^\perp
    u_Fperp_F = u[F_perp, F]  # Bloc reliant F^\perp et F
    
    # Calcul de l'inverse des blocs
    u_F_inv = schur_inverse(u_F)
    S = u_F_perp - u_Fperp_F * u_F_inv  * u_F_Fperp  # Complément de Schur
    inv_S = schur_inverse(S)
    
    # Calcul de l'inverse restreint de u sur F
    u_inv_F = u_F_inv + u_F_inv * u_F_Fperp * inv_S * u_Fperp_F * u_F_inv
    
    return u_inv_F
end


function compute_full_inverse(u)
    n = size(u, 1)
    full_inverse = zeros(n, n)
    
    # Construire l'inverse en explorant toutes les décompositions possibles de F et F_perp
    for i in 1:n
        for j in i+1:n
            F = [i, j]
            F_perp = setdiff(1:n, F)  # Correction pour obtenir F_perp correctement
            
            if  length(F_perp) > 0
                inv_F = compute_inverse_restricted(F, F_perp, u)
                #println( i , ", " ,  j , ", " , F , ", " , F_perp, ", " , inv_F )
                full_inverse[i, i] = inv_F[1,1]  # Extraction correcte
                full_inverse[j, j] = inv_F[2,2]  # Extraction correcte
                
                full_inverse[i, j] = inv_F[1,2]  # Extraction correcte
                full_inverse[j, i] = full_inverse[i, j]  # Remplissage symétrique
            else 
                
                return  compute_inverse_restricted(F, F_perp, u)   
                    
            end
        end
    end
    
    return full_inverse
end





# Exemple d'utilisation
u = [4.0 1.0 0.5; 1.0 3.0 0.2; 0.5 0.2 2.0]  # Matrice symétrique
n = 1
A = randn(n, n)  # Matrice aléatoire avec des entrées normales (ou rand(n, n) pour des entrées entre 0 et 1)
u = 0.5 * (A + A') 
u_inv = inv(u)  # Calcul de l'inverse complet de u
computed_inv1 = compute_full_inverse(u)  # Reconstruction de u^{-1}
computed_inv2 = schur_inverse(u)  # Reconstruction de u^{-1}



# Vérification : computed_inv doit être proche de u_inv
println("Inverse calculé de u:")
#println(computed_inv)
println("Inverse réel de u:")
#println(u_inv)
println("Erreur:")
#println(norm(computed_inv - u_inv))
println(norm(computed_inv1 - u_inv)/norm(u_inv))
println(norm(computed_inv2 - u_inv)/norm(u_inv))


Inverse calculé de u:
Inverse réel de u:
Erreur:
1.0
0.0


In [124]:
using Plots

function schur_inverse(A::Matrix{Float64})
    n = size(A, 1)
    if n == 1 
        return [1.0 / A[1, 1]]
    elseif n == 0 
        return 1 
    end 
    
    k = div(n, 2)  # Division en blocs
    A11 = A[1:k, 1:k]
    A12 = A[1:k, k+1:end]
    A21 = A[k+1:end, 1:k]
    A22 = A[k+1:end, k+1:end]

    inv_A11 = schur_inverse(A11)
    S = A22 - A21 * inv_A11 * A12  # Complément de Schur
    inv_S = schur_inverse(S)

    # Reconstruction de l'inverse
    inv_A = [inv_A11 + inv_A11 * A12 * inv_S * A21 * inv_A11   -inv_A11 * A12 * inv_S
             -inv_S * A21 * inv_A11                             inv_S]
    
    return inv_A
end


function compute_full_inverse(u)
    n = size(u, 1)
    full_inverse = zeros(n, n)

    #p = heatmap(full_inverse, color=:viridis, title="Matrice U")
    #display(p)  # Forcer l'affichage

    if n == 1 
        return  compute_inverse_restricted([1], [], u)      
    end 

    full_inverse0 = copy(full_inverse)

    # Exploration des paires (F, F_perp)
    for i in 1:n
        for j in i+1:n
            F = [i, j]
            F_perp = setdiff(1:n, F)  # Définition correcte de F_perp
            #println("F,  F_perp :", F, F_perp )

            inv_F = compute_inverse_restricted(F, F_perp, u)

            full_inverse[F, F] .= inv_F  # Remplissage correct du bloc
            #full_inverse[i, i] = inv_F[1,1]  # Remplissage correct du bloc
            #full_inverse[j, j] = inv_F[2,2]  # Remplissage correct du bloc
            #full_inverse[i, j] = inv_F[1,2]  # Remplissage correct du bloc
            #full_inverse[j, i] = full_inverse[i, j]  # Remplissage correct du bloc
            #println(inv_F , full_inverse)
            # Affichage forcé au milieu du code
            #p = heatmap(full_inverse - full_inverse0, color=:viridis, title="Matrice U")
            #display(p)  # Forcer l'affichage
            #println(full_inverse - full_inverse0)
            full_inverse0 = copy(full_inverse)        

        end
    end

    return full_inverse
end

n = 10
A = randn(n, n)  
u = 0.5 * (A + A')  # Matrice symétrique
I_n = Matrix{Float64}(I, n, n)  # Identité explicite

u_inv = inv(u)  # Inverse réelle
computed_inv1 = compute_full_inverse(u)  # Approche via complément de Schur
computed_inv2 = schur_inverse(u)  # Approche récursive complète

# Vérification des erreurs relatives
err1 = norm(computed_inv1 - u_inv) / norm(u_inv)
err1_left = norm(computed_inv1 * u - I_n) / norm(I_n)
err1_right = norm(u * computed_inv1 - I_n) / norm(I_n)

err2 = norm(computed_inv2 - u_inv) / norm(u_inv)
err2_left = norm(computed_inv2 * u - I_n) / norm(I_n)
err2_right = norm(u * computed_inv2 - I_n) / norm(I_n)

println("Erreur relative méthode 1 : $err1, $err1_left, $err1_right")
println("Erreur relative méthode 2 : $err2, $err2_left, $err2_right")


Erreur relative méthode 1 : 1.7736759005408366e-8, 6.960548474231698e-8, 6.955983200926062e-8
Erreur relative méthode 2 : 9.991521436446022e-16, 1.885291236625717e-15, 1.70742146412097e-15
